# Interpretabilidade e explicabilidade

Os alunos deverão explorar algum método de interpretação de modelos e/ou modelos naturalmente interpretáveis (por exemplo, árvores de decisão), a fim de compreender ou extrair hipóteses sobre quais atributos são aparentemente mais relevantes para a tarefa de predição e/ou como eles impactam na decisão do modelo. Sugere-se que se faça esta investigação apenas para um modelo, a ser escolhido com base no melhor desempenho preditivo em dados de teste (isto é, o melhor modelo conforme análise do grupo). Os grupos deverão incluir no relatório informações obtidas desta análise, como gráficos, tabelas, etc, e discutir a respeito das relações encontradas na análise que mais chamaram a atenção, seja pela pertinência da associação ou por ser um resultado inesperado.

## Setup

In [ ]:
%pip install pandas plotly matplotlib seaborn scikit-learn shap

In [ ]:
# Módulo para leitura e manipulação dos dados
import pandas as pd

# Módulo para manipulação de arrays e matrizes
import numpy as np

# Módulos para visualização de dados e plotagem de gráficos
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

# Módulos específicos da sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, mutual_info_classif

from sklearn.inspection import permutation_importance

# Módulo para balanceamento de classes
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

import shap

https://machinelearningmastery.com/calculate-feature-importance-with-python/

You need to be using this version of scikit-learn or higher.
0.22.1

In [ ]:
# check scikit-learn version
import sklearn
print(sklearn.__version__)

## Carregamento dos dados e configuração do modelo

Dataset pré-processado no notebook [t1-spot-checking.ipynb](./t1-spot-checking.ipynb)

---
Dataset obtido em https://www.kaggle.com/datasets/thedevastator/higher-education-predictors-of-student-retention/data

Original: https://zenodo.org/records/5777340#.Y7FJotJBwUE

In [ ]:
data = pd.read_csv("./data/clean-dataset.csv")
X = data.drop('Target', axis=1)
y = data['Target']
print(X.shape, y.shape)

In [ ]:
# Algoritmos selecionados para otimização de hiperparâmetros

lr = LogisticRegression()
abc = AdaBoostClassifier()
svmachine = svm.SVC(probability=True)

algo_dict = {'Logistic Regression': lr, 'AdaBoost': abc, 'SVM': svmachine}

# Treinando o modelo em todo o dataset

# 2. Interpretabilidade intrínseca

**Interpretabilidade:** a capacidade de explicar ou apresentar em termos compreensíveis para um ser humano.

# 3. Interpretabilidade extrínseca

**Explicabilidade:** coleção de artefatos visuais e/ou interativos que fornecem ao usuário uma descrição suﬁciente do comportamento de
um modelo para executar com precisão tarefas como avaliação, conﬁança, previsão ou melhoria de um modelo.

**Explainable Machine learning:** análises post-hoc e técnicas usadas para entender os mecanismos para tomada de decisão ou as previsões realizadas pelo modelo treinado.

In [ ]:
def make_pipeline(model):
    steps = list()

    steps.append(('Feature Selection', SelectKBest(k=4, score_func=mutual_info_classif)))
    steps.append(('Normalização', StandardScaler()))
    steps.append(('Balanceamento da classe minoritária', SMOTE(sampling_strategy='minority')))
    steps.append(('Modelo', model))

    # Cria a pipeline
    pipe = Pipeline(steps=steps)

    return pipe

Carrega os hiperparâmetros selecionados

In [ ]:
import ast

optimal_hyperparams = pd.read_csv("./data/otimizacao-hiperparams.csv")
result = optimal_hyperparams.iloc[2]
best_params = result.iloc[0]
display(optimal_hyperparams)

# Convert the string representation of the dictionary to an actual dictionary
best_params_dict = ast.literal_eval(best_params)
print(best_params_dict)

Treina para todos os dados

In [ ]:
model = make_pipeline(model=algo_dict['AdaBoost'])
model.set_params(Modelo__n_estimators = best_params_dict['Modelo__n_estimators'],
                         Modelo__learning_rate = best_params_dict['Modelo__learning_rate'])

model.fit(X, y)

selected_features = model.named_steps['Feature Selection'].get_support(indices=True)
print("Selected features:", selected_features)
selected_feature_names = [data.columns[i] for i in selected_features]
print("Selected feature names:", selected_feature_names)

## Importância de Atributos

- Método de importância de atributos com **testes de permutação é model agnostic**
  - https://machinelearningmastery.com/calculate-feature-importance-with-python/
  - https://scikit-learn.org/stable/modules/permutation_importance.html

Note: Your results may vary given the stochastic nature of the algorithm or evaluation procedure, or differences in numerical precision. **Consider running the example a few times and compare the average outcome.**

Questões
- Devemos usar pipeline?
  - https://stackoverflow.com/questions/62106204/permutation-importance-using-a-pipeline-in-scikit-learn
- Como importar os hiperparâmetros otimizados?

In [ ]:
column_indices = {column: index for index, column in enumerate(data.columns)}
print(column_indices)

In [ ]:
# perform permutation importance
results = permutation_importance(model, X, y, scoring='f1')
importance = results.importances_mean

# summarize feature importance
for i,v in enumerate(importance):
	print('Feature: %0d, Score: %.5f' % (i,v))

# plot feature importance
ax = plt.barh([x for x in range(len(importance))], importance)

# Show the major grid and style it slightly.
plt.grid(which='major', color='#DDDDDD', linewidth=0.8)
# Show the minor grid as well. Style it in very light gray as a thin,
# dotted line.
plt.grid(which='minor', color='#EEEEEE', linestyle=':', linewidth=0.5)

# Fixa o eixo x do gráfico
plt.xlim(-0.05, max(importance) + 0.1)
plt.yticks(ticks=range(len(importance)), labels=[column for column in list(column_indices.keys())[:len(importance)]], rotation=0)

plt.show()


## SHAP

https://github.com/shap/shap


Usando o KernelExplainer, por ser mais agnóstico a modelos, pois o pacote não suporta o AdaBoost diretamente. https://shap.readthedocs.io/en/latest/example_notebooks/tabular_examples/model_agnostic/Simple%20Kernel%20SHAP.html#Using-KernelExplainer

In [ ]:
# Transform the data using the pipeline's feature selection and scaling steps
X_transformed = pd.DataFrame(
    model.named_steps['Normalização'].transform(model.named_steps['Feature Selection'].transform(X)),
    columns=selected_feature_names
)

# # Pega 100 amostras
# X_transformed = shap.sample(X_transformed, 100)

# Use SHAP to explain the model's predictions
explainer = shap.KernelExplainer(model.named_steps['Modelo'].predict, X_transformed)
shap_values = explainer.shap_values(X_transformed)

# Plot the SHAP values
shap.summary_plot(shap_values, X_transformed)

shap.plots.bar(shap_values)

In [ ]:
for name, _ in algo_dict.items():
    metrics_per_model = metrics_df[metrics_df['Algoritmo'] == name]

    mean = float(round(metrics_per_model['f1'].mean(),3))
    std = float(round(metrics_per_model['f1'].std(),3))
    print(f'F1-Score do {name}: ', mean, f'({std})')

    mean = float(round(metrics_per_model['precision'].mean(),3))
    std = float(round(metrics_per_model['precision'].std(),3))
    print(f'Precisão média do {name}: ', mean, f'({std})')

    mean = float(round(metrics_per_model['recall'].mean(),3))
    std = float(round(metrics_per_model['recall'].std(),3))
    print(f'Recall média do {name}: ', mean, f'({std})')

    mean = float(round(metrics_per_model['roc_auc'].mean(),3))
    std = float(round(metrics_per_model['roc_auc'].std(),3))
    print(f'ROC AUC média do {name}: ', mean, f'({std})')

    print('\n')
print('\n')

Demora bastante para usar o KernelExplainer do SHAP

https://github.com/ModelOriented/DALEX/issues/366

## PDP
https://scikit-learn.org/stable/modules/partial_dependence.html

IDP: unlike a PDP, which shows the average effect of the input feature, an ICE plot visualizes the dependence of the prediction on a feature for each sample separately with one line per sample. Due to the limits of human perception, only one input feature of interest is supported for ICE plots. While the PDPs are good at showing the average effect of the target features, they can obscure a heterogeneous relationship created by interactions.


In [ ]:
from sklearn.inspection import PartialDependenceDisplay

# One-way Partial Dependence Plot
# ==================================
features = list(range(20)) # Índices dos atributos que serão plotados

PartialDependenceDisplay.from_estimator(model, X, features, n_cols=3)
plt.gcf().set_size_inches(15, 30)

PartialDependenceDisplay.from_estimator(model, X, features, n_cols=3, kind='both')
plt.gcf().set_size_inches(15, 30)

# Two-way Partial Dependence Plot
# ==================================

# Grade 2nd sem vs. Scholarship holder
features = [(13,15)]
PartialDependenceDisplay.from_estimator(model, X, features)

# Tuition Fees Up to Date vs. Grade 2nd sem
features = [(11,15)]
PartialDependenceDisplay.from_estimator(model, X, features)